## 作業

Hint: 人物原始邊框座標 (60, 40), (420, 510)

請根據 Lena 圖做以下處理

- 對明亮度做直方圖均衡處理
- 水平鏡像 + 縮放處理 (0.5 倍)
- 畫出人物矩形邊框

In [1]:
import cv2
import numpy as np

# Using lena.png
img = 'img/lena.png'

# default read as cv2.IMREAD_COLOR(彩色圖)
img_bgr = cv2.imread(img, cv2.IMREAD_COLOR)

In [3]:
# Drawing Rectangle :
img_rect = img_bgr.copy()
# cv2.rectangle(original image, rect-left-up(x,y), rect-right-down(x,y), rect-color, rect-thickenss)
start_point = (60, 40) 
end_point = (420, 510) 
color = (0, 0, 255)    
thickness = 3          
cv2.rectangle(img_rect, start_point, end_point, color, thickness)

while True :
    # display of img_rect, named as 'image with rect'
    cv2.imshow('image with rect', img_rect)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [4]:
# Drawing Line :
img_line = img_bgr.copy()
# cv2.rectangle(original image, line-left-up(x,y), line-right-down(x,y), line-color, line-thickenss)
start_point = (60, 40) 
end_point = (420, 510)
color = (0, 0, 255)    
thickness = 3          
cv2.line(img_line, start_point, end_point, color, thickness)

while True :
    # display of img_line, named as 'image with line'
    cv2.imshow('image with line', img_line)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [6]:
# Drawing Text :
img_text = img_bgr.copy()
# cv2.rectangle(original image, text, text_coordinate(x,y), font, font_size, text-color, text-thickenss)
text = '(64, 40)' 
text_coordinate = (64, 40)
font = 0
font_size = 1
color = (255, 0, 0)    
thickness = 2          
cv2.putText(img_text, text, text_coordinate, font, font_size, color, thickness)

while True :
    # display of img_text, named as 'image with text'
    cv2.imshow('image with text', img_text)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [2]:
img_equal_rect = img_bgr.copy()

# transfer color space to HSV image (3D array)
img_equal_rect = cv2.cvtColor(img_equal_rect, cv2.COLOR_BGR2HSV)
# histogram equalization by value(brightness) channel of HSV image (2D array) -> 增強對比
img_equal_rect[:,:,2] = cv2.equalizeHist(img_equal_rect[:,:,2])

# transfer color space return to BGR
img_equal_rect = cv2.cvtColor(img_equal_rect, cv2.COLOR_HSV2BGR)

# cv2.rectangle(original image, line-left-up(x,y), line-right-down(x,y), line-color, line-thickenss)
start_point = (60, 40) 
end_point = (420, 510) 
color = (0, 0, 255)    
thickness = 3          
cv2.rectangle(img_equal_rect, start_point, end_point, color, thickness)

# horizontal flip(水平翻轉) -> 矩形框跟著水平翻轉
# img_bgr[h, w, channel], w reversed which means horizontal flip
img_equal_rect = img_equal_rect[:, ::-1, :]

# resize to 50% of original image -> 矩形框跟著resize
img_equal_rect = cv2.resize(img_equal_rect, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

while True :
    # display of img_equal_rect, named as 'img equal then rect & mirror & zoom'
    cv2.imshow('img equal then rect & mirror & zoom', img_equal_rect)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break